In [1]:
#Importing Libraries
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Load the dataset
df=pd.read_csv("drive/My Drive/CSV_Files/DrugsDotCom_preprocessed.csv",encoding="ISO-8859-1")
df.head()

,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized
0,0,0,Liraglutide,Obesity,""" began with the minimum 0.6 mg dose of Saxend...",10,"March 18, 2017",28,[ began minimum mg dose saxenda february incre...,"['[', 'began', 'minimum', 'mg', 'dose', 'saxen...",[ began minimum mg dose saxenda february incre...
1,1,1,Quetiapine,Bipolar Disorde,""" Caused depression and negative, self defeati...",1,"August 25, 2016",2,[ caused depression negative self defeating th...,"['[', 'caused', 'depression', 'negative', 'sel...",[ caused depression negative self defeating th...
2,2,2,Ethinyl estradiol / levonorgestrel,Birth Control,""" Ever since I started taking this birth contr...",1,"October 30, 2017",2,[ ever since started taking birth control noti...,"['[', 'ever', 'since', 'started', 'taking', 'b...",[ ever since started taking birth control noti...
3,3,3,Lorazepam,Anxiety,""" Had a sort of nervous breakdown during a tri...",9,"June 4, 2014",64,[ sort nervous breakdown trip could catch brea...,"['[', 'sort', 'nervous', 'breakdown', 'trip', ...",[ sort nervous breakdown trip could catch brea...
4,4,4,Aczone,Acne,""" I am 28, have struggled with acne for 10 yea...",10,"June 2, 2015",16,[ struggled acne year product kept consistentl...,"['[', 'struggled', 'acne', 'year', 'product', ...",[ struggled acne year product kept consistentl...


In [4]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [5]:
#Droping duplicates
df=df.drop_duplicates(subset=["drugName","condition","review","rating"],keep='first')

In [6]:
#resertting the index
df = df.copy()
df.reset_index(inplace = True)

In [7]:
df1 = pd.DataFrame(df.groupby('drugName')['review'].apply(' '.join).reset_index())
df1.tail()

,drugName,review
2330,Zyrtec-D,"""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried."""
2331,Zyvox,"""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti..."
2332,ZzzQuil,"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z..."
2333,femhrt,"""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all..."
2334,pHisoHex,"""Before Phisohex could only be purchased by prescription, I used it regullarly for feminine hygenine. The product worked perfect, was long lasting, and I never had any side affects. I used Phisohex from 1966 until, much to my dismay, it was taken off my druggist over the counter shelves and could only be purchased with a prescription. Would still be using it if I could. Wonderful solution!"""


In [8]:
df1['review_wordcount'] = df1['review'].str.split().str.len()

In [9]:
sorted = df1.sort_values('review_wordcount', ascending=False)
sorted.head()

,drugName,review,review_wordcount
1514,Nexplanon,""" I have been on this for 2 1/2 years now. I am going to have it removed because I have gained weight using it, my periods are heavier and longer with clots, I have become very moody, anxious and angry at the very slightest thing which I wasnt before. I get very terrible migraines with aura. I spoke to my doctor and she said she does not think these side effects could be due to the nexplanon. ...",267069
854,Ethinyl estradiol / norethindrone,""" Im on my 2nd round of necon 1/35. I started taking it because the doctor thinks I have endometriosis, but no one is really sure. My periods are extremely painful and I lose my appetite completely. The side effects of necon are the same as symptoms of pregnancy which is kind of alarming. Im also even more depressed and anxious than I was before I started taking it. I take it without the place...",186537
1248,Levonorgestrel,""" I took levonorgestrel under the brand Postinor 2 after an incident with my boyfriend. My boyfriends semen got over my vaginal area and some of it did get inside me (shallow penetration) and this was during my ovulation period. Took the first pill in 43h, second pill 12h later (postinor requires you to eat 2 pills 12h apart). 1 week after taking the pills I had a lot of vaginal discharge and ...",158176
1940,Sertraline,"""(Sertraline) is the most worst depression treatment for my son. The first time when my son had started to take these pills he was feeling normal but one day he started to have unusual pains which he couldnt describe in the chest which was everyday. Because of the pain he was going to A&amp;E for a check up twice in a week and shockingly nothing had came up. But that pain took my precious sons...",149151
855,Ethinyl estradiol / norgestimate,""" Im 19 years old and started taking this birth control after my first one worked perfectly for a few months, until I began to get my period every 8 days. Ive been on this pill for about two weeks and I already hate it. Im constantly moody, depressed and over emotional. Its effecting my relationship between my family, friends and boyfriend. Doesnt help with acne, Ive never had acne till I bega...",147830


In [10]:
df2 = df.groupby('drugName')['rating'].mean().reset_index()
df2.tail()

,drugName,rating
2330,Zyrtec-D,9.000000
2331,Zyvox,8.714286
2332,ZzzQuil,2.000000
2333,femhrt,6.500000
2334,pHisoHex,10.000000


In [11]:
df3 = pd.DataFrame(df.groupby('drugName')['usefulCount'].mean().round().reset_index())
df3.tail()

,drugName,usefulCount
2330,Zyrtec-D,13.0
2331,Zyvox,21.0
2332,ZzzQuil,7.0
2333,femhrt,14.0
2334,pHisoHex,0.0


In [12]:
drug_condition = pd.DataFrame(df.groupby('drugName')['condition'].apply(lambda x: list(np.unique(x)))).reset_index()
drug_condition.tail()

,drugName,condition
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]"
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]"
2332,ZzzQuil,[Insomnia]
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]"
2334,pHisoHex,"[Skin Disinfection, Preoperative]"


In [13]:
import nltk
nltk.download('punkt')
df4 = pd.DataFrame(df.groupby('drugName')['condition'].apply(' '.join).reset_index())
df4.head()
df4['condition'] = df4.apply(lambda row: nltk.word_tokenize(row['condition']), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
data = pd.concat([df1.drugName,drug_condition.condition,df1.review, df2.rating, df3.usefulCount, df1.review_wordcount], axis = 1)
data.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263
2334,pHisoHex,"[Skin Disinfection, Preoperative]","""Before Phisohex could only be purchased by prescription, I used it regullarly for feminine hygenine. The product worked perfect, was long lasting, and I never had any side affects. I used Phisohex from 1966 until, much to my dismay, it was taken off my druggist over the counter shelves and could only be purchased with a prescription. Would still be using it if I could. Wonderful solution!""",10.000000,0.0,67


In [15]:
data1=data.iloc[:100,:]

In [16]:
data1.head()

,drugName,condition,review,rating,usefulCount,review_wordcount
0,A + D Cracked Skin Relief,[Bacterial Skin Infection],"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore.""",10.00000,6.0,63
1,A / B Otic,[Otitis Media],"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief.""",10.00000,19.0,73
2,Abacavir / lamivudine,[HIV Infection],"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ...",10.00000,4.0,178
3,Abacavir / lamivudine / zidovudine,[HIV Infection],"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since.""",9.00000,1.0,41
4,Abatacept,"[Juvenile Idiopathic Arthritis, Rheumatoid Arthritis]","""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou...",7.26087,48.0,1603


In [17]:
data2=data.iloc[-100:-1,:]

In [18]:
data

,drugName,condition,review,rating,usefulCount,review_wordcount
0,A + D Cracked Skin Relief,[Bacterial Skin Infection],"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore.""",10.000000,6.0,63
1,A / B Otic,[Otitis Media],"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief.""",10.000000,19.0,73
2,Abacavir / lamivudine,[HIV Infection],"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ...",10.000000,4.0,178
3,Abacavir / lamivudine / zidovudine,[HIV Infection],"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since.""",9.000000,1.0,41
4,Abatacept,"[Juvenile Idiopathic Arthritis, Rheumatoid Arthritis]","""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou...",7.260870,48.0,1603
5,Abilify,"[3</span> users found this comment helpful., Agitated State, Autism, Bipolar Disorde, Borderline Personality Disorde, Depression, Major Depressive Disorde, Obsessive Compulsive Disorde, Post Traumatic Stress Disorde, Schizoaffective Disorde, Schizophrenia, Tourette's Syndrome]","""10 months on 5mg abilify + wellbutrin and lithium. Gained 15-18 pounds. Found out this week that Abilify causes weight gain and its the culprit - not an injury Id been blaming it on. And Id blamed myself and started &quot;disordered eating&quot; out of desperation. It gets rid of my mood swings and crying for no reason. I feel generally good except foggy while at work where I do not enjoy bei...",6.453875,40.0,37040
6,Abiraterone,[Prostate Cance],"""Began Zytiga with Prednisone 4/23/14 when considered hormone resistant. Highest PSA 4/1/14: 214. Lowest PSA 3/30/15 6.0. Most recent PSA 5/4/15 8.6. Added Metformin 3/3/15. Also Lupron every three months and Xgeva monthly. Very few side effects."" ""effectiveness lasted 8 months side effects unpleasant"" ""Have taken Zytiga for a year now, one pill each morning with light breakfast since I have d...",9.100000,38.0,274
7,Acamprosate,[Alcohol Dependence],"""35 years old...1 week in...Im not experiencing the fatigue and nausea associated w/ naltrexone. No desire to drink--I feel like something is wrong. I wish I was prescribed this ten years ago."" ""40 years of heavy drinking until severe depression caused me to seek help. With the help of my psychologist and Campral I live again. A few minor gut type side effects but totally worth it."" ""A miracle...",8.792000,63.0,10503
8,Acanya,[Acne],"""Acanya is awesome. I had a pizza face and my mom got me some to use; my acne started clearing up in a few hours, and now a week later my skin is completely clear."" ""Also for the people saying it dries our your skin ver

In [19]:
data2.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount
2329,Zyrtec,"[7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria]","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263


In [20]:
data200=pd.concat([data1,data2])

In [21]:
data200.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount
2329,Zyrtec,"[7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria]","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263


In [22]:
data200.head()

,drugName,condition,review,rating,usefulCount,review_wordcount
0,A + D Cracked Skin Relief,[Bacterial Skin Infection],"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore.""",10.00000,6.0,63
1,A / B Otic,[Otitis Media],"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief.""",10.00000,19.0,73
2,Abacavir / lamivudine,[HIV Infection],"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ...",10.00000,4.0,178
3,Abacavir / lamivudine / zidovudine,[HIV Infection],"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since.""",9.00000,1.0,41
4,Abatacept,"[Juvenile Idiopathic Arthritis, Rheumatoid Arthritis]","""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou...",7.26087,48.0,1603


In [23]:
from textblob import TextBlob
data200['polarity'] = data200['review'].apply(lambda x: TextBlob(x).sentiment.polarity)
data200.head()

,drugName,condition,review,rating,usefulCount,review_wordcount,polarity
0,A + D Cracked Skin Relief,[Bacterial Skin Infection],"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore.""",10.00000,6.0,63,0.284722
1,A / B Otic,[Otitis Media],"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief.""",10.00000,19.0,73,0.193750
2,Abacavir / lamivudine,[HIV Infection],"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ...",10.00000,4.0,178,0.181250
3,Abacavir / lamivudine / zidovudine,[HIV Infection],"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since.""",9.00000,1.0,41,-0.055556
4,Abatacept,"[Juvenile Idiopathic Arthritis, Rheumatoid Arthritis]","""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou...",7.26087,48.0,1603,0.096676


In [24]:
import string
def Effective_Review(Review):
    Pos_Word=[]
    Review=Review.split(".")
    for word in Review:
        tb = TextBlob(word)
        if tb.sentiment.polarity >= 0.8:
            Pos_Word.append(word)
    return Pos_Word

Effective = lambda x: Effective_Review(x)

In [25]:
data200['Pos_Words'] = data200.review.apply(Effective)
data200.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount,polarity,Pos_Words
2329,Zyrtec,"[7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria]","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795,0.091117,"[ Felt good! ITCHING STOPPED, No drowsiness is the best part, I give Zyrtec 8 stars BC it is a great otc fix for allergies, feeling great, Great product, I am happy to say, Excellent product, "" ""Ive had chronic urticaria for over 2 years now, and out of all the allergy medicines, Zyrtec works the best, "" ""It works great for the symptoms of my allergies and hay fever, and nothing was work..."
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,[]
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"[ If this works then its an amazing, wonderful drug]"
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,[]
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263,0.067333,"["" ""Femhrt worked wonderful, Works great]"


In [26]:
def Not_Effective_Review(Review):
    Neg_Word=[]
    Review=Review.split(".")
    for word in Review:
        tb = TextBlob(word)
        if tb.sentiment.polarity <= -0.8:
            Neg_Word.append(word)
    return Neg_Word

Not_Effective = lambda x: Not_Effective_Review(x)

In [27]:
data200['Neg_Words'] = data200.review.apply(Not_Effective)

In [28]:
data200.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount,polarity,Pos_Words,Neg_Words
2329,Zyrtec,"[7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria]","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795,0.091117,"[ Felt good! ITCHING STOPPED, No drowsiness is the best part, I give Zyrtec 8 stars BC it is a great otc fix for allergies, feeling great, Great product, I am happy to say, Excellent product, "" ""Ive had chronic urticaria for over 2 years now, and out of all the allergy medicines, Zyrtec works the best, "" ""It works great for the symptoms of my allergies and hay fever, and nothing was work...","[the welts were painful and then became HORRIBLY insanely itchy, Terrible, I just got done with a horrible case from penicillin, HATE HATE HATE THE PACKAGING, The horrible migraines were withdrawal symptoms! I immediately stopped giving it to my kids and taking it myself, It took a couple weeks to get through the worst of it, but I am still itching after a month]"
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,[],[]
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"[ If this works then its an amazing, wonderful drug]","[ It is a very unpleasant feeling, It was the worst 2 weeks of my life, Weakness, I felt horrible, My doctor keeps on eye on me but the brown tongue and funky taste is disgusting]"
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,[],[]
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263,0.067333,"["" ""Femhrt worked wonderful, Works great]",[it worked well for symptom management but the side effect was horrible]


In [29]:
data200['Pos_Words'] = data200['Pos_Words'].apply(lambda x:', '.join([i for i in x if i not in string.punctuation]))

In [30]:
pip install textacy

In [31]:
import textacy

In [37]:
def Effective_Words(Review):
    Effective_Word=[]
    Review=str(Review)
    Review_text=Review

    Review_doc=textacy.make_spacy_doc(Review_text,lang='en_core_web_sm')
    
    pattern_adv_adj = textacy.extract.matches(Review_doc, patterns=[{"POS" : "ADV"},{"POS" :"ADJ"}])
    pattern_adj_adj = textacy.extract.matches(Review_doc, patterns=[{"POS" : "ADJ"},{"POS" :"ADJ"}])
    pattern_verb_adj= textacy.extract.matches(Review_doc, patterns=[{"POS" : "VERB"},{"POS" :"ADJ"}])
    pattern_adj_adv= textacy.extract.matches(Review_doc, patterns=[{"POS" : "ADJ"},{"POS" :"ADV"}])

    for word in pattern_adv_adj:
       Effective_Word.append(word.text)
    for word in pattern_adj_adj:
       Effective_Word.append(word.text)
    for word in pattern_verb_adj:
       Effective_Word.append(word.text)
    for word in pattern_adj_adv:
       Effective_Word.append(word.text)
    return Effective_Word

Effective_W = lambda x: Effective_Words(x)

In [38]:
data200['Pos_Words2'] = data200.Pos_Words.apply(Effective_W)

In [39]:
def Neg_Effect_Words(Review):
    Neg_Effect_Word=[]
    Review=str(Review)
    Review_text=Review
    
    Review_doc=textacy.make_spacy_doc(Review_text,lang='en_core_web_sm')
    
    pattern_adv_adj = textacy.extract.matches(Review_doc, patterns=[{"POS" : "ADV"},{"POS" :"ADJ"}])
    pattern_adj_adj = textacy.extract.matches(Review_doc, patterns=[{"POS" : "ADJ"},{"POS" :"ADJ"}])
    pattern_verb_adj= textacy.extract.matches(Review_doc, patterns=[{"POS" : "VERB"},{"POS" :"ADJ"}])
    pattern_adj_adv= textacy.extract.matches(Review_doc, patterns=[{"POS" : "ADJ"},{"POS" :"ADV"}])

    for word in pattern_adv_adj:
       Neg_Effect_Word.append(word.text)
    for word in pattern_adj_adj:
       Neg_Effect_Word.append(word.text)
    for word in pattern_verb_adj:
       Neg_Effect_Word.append(word.text)
    for word in pattern_adj_adv:
       Neg_Effect_Word.append(word.text)
    return Neg_Effect_Word

Neg_Effect_W = lambda x: Neg_Effect_Words(x)

In [40]:
data200['Neg_Words2'] = data200.Neg_Words.apply(Neg_Effect_W)

In [41]:
data200.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount,polarity,Pos_Words,Neg_Words,Pos_Words2,Neg_Words2
2329,Zyrtec,"[7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria]","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795,0.091117,"Felt good! ITCHING STOPPED, No drowsiness is the best part, I give Zyrtec 8 stars BC it is a great otc fix for allergies, feeling great, Great product, I am happy to say, Excellent product, "" ""Ive had chronic urticaria for over 2 years now, and out of all the allergy medicines, Zyrtec works the best, "" ""It works great for the symptoms of my allergies and hay fever, and nothing was worki...","[the welts were painful and then became HORRIBLY insanely itchy, Terrible, I just got done with a horrible case from penicillin, HATE HATE HATE THE PACKAGING, The horrible migraines were withdrawal symptoms! I immediately stopped giving it to my kids and taking it myself, It took a couple weeks to get through the worst of it, but I am still itching after a month]","[feeling great, had chronic, works great, m happy, works great, Works great]",[]
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,,[],[],[]
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"If this works then its an amazing, wonderful drug","[ It is a very unpleasant feeling, It was the worst 2 weeks of my life, Weakness, I felt horrible, My doctor keeps on eye on me but the brown tongue and funky taste is disgusting]",[],"[very unpleasant, felt horrible]"
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,,[],[],[]
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263,0.067333,""" ""Femhrt worked wonderful, Works great",[it worked well for symptom management but the side effect was horrible],[worked wonderful],[]
